# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [ ]:
import xlwings as xw 
import numpy as np
wb1 = xw.Book('себестоимостьА_в1.xlsx')
sht = wb1.sheets['Рецептура']
consumption = sht.range('G7:O10').options(np.array).value
consumption

In [ ]:
unit_price = sht.range('G14:O14').options(np.array).value
unit_price

In [ ]:
cost_price = np.nan_to_num(consumption*unit_price).sum(axis = 1)
cost_price

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [ ]:
sht.range('T7:T10').options(transpose= True).value = cost_price
sht.range('T6').value = 'Себестоимость'

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

In [ ]:
sht['T4:T6'].merge()
sht['B3:T3'].merge()
sht['B3:T3'].api.Borders.Weight= 2
head = sht['T4:T6']
head.value = 'Себестоимость'
head.autofit()
head.font.color = (255,255,255)
head.color = (230, 230, 250)
head.font.bold = True
head.api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter
head.api.VerticalAlignment = xw.constants.HAlign.xlHAlignCenter
head.api.Borders.Weight = 2
sht['T7:T10'].color = (50, 205, 50)
sht['T7:T10'].api.Borders.Wight = 2

4. Выполнить 3 с помощью "протягиваемых" формул.

In [ ]:
sht['T7:T10'].value = '=SUMPRODUCT(G7:O7; G14:O14)'

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [ ]:
import pandas as pd

In [ ]:
recipes = pd.read_csv('recipes_sample.csv')
recipes = recipes[['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients']]
reviews = pd.read_csv('reviews_sample.csv')

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [ ]:
recipes_s =  recipes.sample(frac = 0.05)
reviews_s = reviews.sample(frac = 0.05)
with pd.ExcelWriter('recipes.xlsx') as writer: 
    recipes_s.to_excel(writer, sheet_name = 'Рецепты')
    reviews_s.to_excel(writer, sheet_name = 'Отзывы')

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [ ]:
import xlwings as xw

In [ ]:
wb = xw.Book('recipes.xlsx')
sht1 = wb.sheets['Рецепты']
sht1['H1'].value = 'second_assign'
sht1['H2:H1501'].value = (recipes_s['minutes'].values * 60)[:, None]

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [ ]:
sht1['I1'].value= 'seconds_formula'
sht1['I2:I1501'].formula = '=D2*60'

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [ ]:
sht1['H1:I1'].font.bold = True

In [ ]:
from openpyxl.styles.alignment import Alignment
sht1['H1:I1'].alignment = Alignment(horizontal="center")

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [ ]:
for i in sht1['D2'].expand('down'):
    if i.value < 5: 
        i.color = (124, 252, 0)
    elif 5<=i.value<=10:
        i.color = (255,255,0)
    else: 
        i.color = (255,0,0)

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [ ]:
sht1['J1'].value = 'n_reviews'
fml = sht1['J1'].formula = '=COUNTIF(Отзывы!D2:D6336, B2)'
sht1.range('J2:J1501').formula = fml

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
sht2 = wb.sheets['Отзывы']
def validate():
    for i in range(2, reviews_s.shape[0] + 1):
        if sht2.range(f'C{i}').value not in recipes_s['id']:
            sht2.range(f'A{i}: F{i}').color = (220, 151, 255)
        elif sht2.range(f'E{i}').value not in range(0,6):
            sht2.range(f'A{i}:F{i}').color = (255, 0, 0)
validate()

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

In [ ]:
import csv 
with open('recipes_model.csv', encoding = 'utf-8') as file:
    df = csv.reader(file)
    wb = xw.Book()
    wb.save('recipes_model.xlsx')
    sht1 = wb.sheets[0]
    sht1.name = 'Модель'
    sht1['A2'].value = list(map(lambda x: x.split('\t'), list(file)))

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
last_row = sht1['A2'].expand('down').size+1
sht1[f"J2:J{last_row}"].value = '=TEXTJOIN(" ";;B2;UPPER(C2); IFS(G2="FK"; TEXTJOIN(" "; ;"REFERENCES";H2; SUBSTITUTE("(%)";"%";I2)); G2="PK";"PRIMARY KEY"; TRUE; ""); IF(AND(D2="Y";G2<>"PK");"NOT NULL";""))'

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
sht1[f'A1:J{last_row}'].value = ['Сущность', 'Атрибут', 'Тип данных', 'NOT NULL']
sht1[f'A1:J{last_row}'].columns.autofit()
sht1[f'A1:J1'].color = '#00ccff'
sht1[f'A1:J1'].font.bold = True
#autofilt
sht1.tables.add(sht1.used_range, name = 'a_table')
sht1.tables['a_table'].show_autofilter = True

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 

In [ ]:
try:
    wb.sheets.add('Статистика')
except: 
    pass
finally:
    stats_sheet = wb.sheets['Статистика']
model = sht1['A1:B18'].value
model_table = pd.DataFrame(model[1:], columns = model[0])
grouped_data= model_table.groupby('Сущность').count()
stats_sheet['A1'].value = grouped_data
figure = grouped_data.plot.bar().get_figure()